In [47]:
# gerekli kütüphaneler
!pip install -q pandas numpy google-genai sentence-transformers faiss-cpu

import pandas as pd
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from google import genai
from google.colab import userdata




In [48]:
# API anahtarı, client tanımı
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    client = genai.Client(api_key=API_KEY)
    print("API Client oluşturma başarılı.")
except Exception as e:
    print(f"Hata: Client başlatılamadı.. Detay: {e}")
    client = None

# Dosya adı
file_path = 'train_expanded.json'

data_list = []
try:
    # hatalı satırları atlama
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    data_list.append(json.loads(line))
                except json.JSONDecodeError:
                    continue

    # DataFrame'e dönüştürme, tekrar edenlerin temizliği
    df = pd.DataFrame(data_list)
    initial_count = len(df)
    df = df.drop_duplicates(subset=['question', 'answer']).reset_index(drop=True)

    print(f"Başlangıç: {initial_count} Soru. Temizleme Sonrası: {len(df)} Soru.")

    # Vektörleştirme; soru cevabı birleştirme
    df['combined_text'] = 'Soru: ' + df['question'] + ' Cevap: ' + df['answer']


except FileNotFoundError:
    print(f"HATA: '{file_path}' dosyası bulunamadı..")
    raise

API Client oluşturma başarılı.
Başlangıç: 200 Soru. Temizleme Sonrası: 89 Soru.


**Bu kısımda embedding modeli değiştirmek gerekti çünkü sorgulanan verileri doğru görüp getiremedi.**

In [49]:




print("\nEmbedding modeli yükleniyor: paraphrase-multilingual-mpnet-base-v2...")
embedder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# vektörleştirme
print(f"{len(df)}vektörlere dönüştürülüyor...")
corpus_embeddings = embedder.encode(df['combined_text'].tolist(), show_progress_bar=True)

# faiss indexi, verileri ekleme
embedding_dim = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(corpus_embeddings)


# indexi kaydetme
faiss_index_file = "faiss_index_faq.bin"
faiss.write_index(index, faiss_index_file)
print(f" '{faiss_index_file}' kaydedildi.")


Embedding modeli yükleniyor: paraphrase-multilingual-mpnet-base-v2...
89vektörlere dönüştürülüyor...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

 'faiss_index_faq.bin' kaydedildi.


In [50]:


model_name = 'gemini-2.5-flash'

def generate_rag_response(user_input, history):
    """
    Gradio arayüzünden gelen kullanıcı girdisini işler

    """
    global client, embedder, index, df

    # hata kontrol
    if client is None:
        return "Chatbot şu anda kullanılamıyor."
    if index is None:
        return "Veri yükleme hatası nedeniyle arama yapılamıyor."



    # Selamlama Filtresi
    greetings = ["merhaba", "selam", "hi", "hello", "günaydın", "iyi günler"]


    user_words = user_input.lower().split()

    is_greeting = any(word in greetings for word in user_words[:2])

    if is_greeting:
        return "Merhaba! Size siparişleriniz, iadeleriniz, hesap bilgileriniz veya diğer destek konuları hakkında nasıl yardımcı olabilirim?"


    # başarılı arama için girdiyi verisetinin diline çevirme
    translation_prompt = f"Translate the following user query to English only: {user_input}"
    try:
        translation_response = client.models.generate_content(
            model=model_name, contents=translation_prompt
        )
        english_query = translation_response.text.strip().split('\n')[0].replace('*', '')
        print(f"[ARAMA SORGUSU] İngilizce çeviri: {english_query}")
    except Exception:
        english_query = user_input # Çeviri hatası durumunda orijinali kullan

    # RAG context veri çekme
    k = 3
    query_embedding = embedder.encode([english_query])
    D, I = index.search(query_embedding, k)
    retrieved_chunks = df.iloc[I[0]]['combined_text'].tolist()
    rag_kaynagi = "\n---\n".join(retrieved_chunks)

    # GEMİNİ PROMPT
    final_prompt = f"""
    SEN bir müşteri destek chatbot'usun. Tüm cevapları KESİNLİKLE TÜRKÇE olarak vermelisin.
    Sadece aşağıdaki KAYNAK BİLGİSİNDE bulunan bilgilere dayanarak, kullanıcının sorusuna kibar ve özlü bir dille cevap ver.



    Eğer Kaynak Bilgide sorunun bir bölümünün cevabı yoksa, **bulduğu kısım hakkında bilgi ver ve bulamadığı kısım için ayrı bir cümleyle "Tarafımda bu konu hakkında bilgi bulunmamaktadır, lütfen destek ekibimizle iletişime geçin." diye yanıtla. Daha sonra KAYNAKTAN konuyla alakalı en yakın  mevcut bilgiyi alıp yaz.**

    KULLANICI SORUSU: {user_input}

     KAYNAK BİLGİ (ALAKALI)
    {rag_kaynagi}
    --------------------------
    """

    # API çağırma
    try:
        response = client.models.generate_content(
            model=model_name,
            contents=final_prompt
        )
        return response.text
    except Exception as e:
        return f"Gemini API hatası: {e}"



In [51]:
# Chatbot Arayüzü

initial_message = [
    {"role": "assistant", "content": "Merhaba! Ben Müşteri Destek Asistanı. Sipariş, iade veya hesap sorunlarınızla ilgili size nasıl yardımcı olabilirim?"}
]
iface = gr.ChatInterface(
    fn=generate_rag_response, # ana fonksiyon
    chatbot=gr.Chatbot(height=300), # sohbet mesajları kutusu yükseklik
    textbox=gr.Textbox(placeholder="Lütfen sorunuzu girin...", container=False, scale=7), # Kullanıcının giriş kutusu
    title="Müşteri Destek Asistanı",
    description="Müşteriler tarafından 'sıkça sorulan sorular'ın cevabını içermektedir.",
    theme="soft" # Arayüz teması
)

# Arayüzü Başlatma

iface.queue().launch(share=True)

/tmp/ipython-input-3184166462.py:8: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=300), # sohbet mesajları kutusu yükseklik
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a558fd74ddaef01a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
